# **Attribution**

Inspiration from Hamza's Course on Enterprise RAG and Multi-Agent Applications

https://maven.com/boring-bot/advanced-llm

# **JINA Embeddings**

In [1]:
import requests
from google.colab import userdata

def get_embeddings(content):
  url = 'https://api.jina.ai/v1/embeddings'
  headers = {
      'Content-Type': 'application/json',
      'Authorization': 'Bearer '+ userdata.get('jina_apikey')
  }
  data = {
      "model": "jina-embeddings-v3",
      "task": "text-matching",
      "late_chunking": False,
      "dimensions": 1024,
      "embedding_type": "float",
      "input": [
          content
      ]
  }

  response = requests.post(url, headers=headers, json=data)
  print(response.json())
  return response.json()

In [5]:
demo_embeddings = get_embeddings("40 bed summer hostel about 3 miles from Gillingham, housed in a districtive converted Oast House in a semi-rural setting.")

{'model': 'jina-embeddings-v3', 'object': 'list', 'usage': {'total_tokens': 33, 'prompt_tokens': 33}, 'data': [{'object': 'embedding', 'index': 0, 'embedding': [-0.109475665, 0.0450993, 0.04877086, 0.028413778, -0.044740826, -0.09973108, -0.09658188, -0.0024917861, -0.042275872, -0.106235676, 0.119427994, 0.0036004714, 0.03112956, -0.010226051, -0.054508995, -0.12750836, -0.024459677, 0.04197465, 0.051268466, 0.07324717, -0.044268936, -0.045601655, -0.0527214, -0.113108054, 0.05448354, 0.088933736, -0.1149054, 0.05819249, 0.057165686, -0.007271633, 0.005998137, 0.07181222, -0.007073966, -0.037292037, -0.014403569, 0.06874381, 0.03427087, 0.005492415, -0.005071177, -0.10698354, -0.038128987, 0.056153454, -0.03349593, 0.06647212, -0.035142135, 0.07637406, 0.041832693, 0.044147275, -0.02233918, 0.014244205, -0.02918599, 0.03154696, 0.054066524, 0.026703231, 0.05361546, 0.059967905, 0.034303285, -0.02122418, -0.056088917, 0.050068278, 0.03408225, -0.04388147, -0.07917087, 0.038208958, -0.0

In [6]:
demo_embeddings.get('data')
data = demo_embeddings.get('data')

if(data is not None):
  embeddings = data[0]['embedding']

In [ ]:
len(embeddings)

1024

In [2]:
!pip install lancedb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 20.0 MB/s eta 0:00:00


In [3]:
import lancedb

In [4]:
import uuid
db = lancedb.connect("/content/lancedb_dir")
table_name = "sample_documents"

documents = [
]

for document in documents:
  embeddings = get_embeddings(document)
  data = embeddings.get('data')

  if(data is not None):
    document["vector"] = data[0]['embedding']

In [54]:
documents

[{'id': 1,
  'text': 'The Eiffel Tower is in Paris.',
  'vector': [-0.009632081,
   -0.03405587,
   0.064153634,
   0.1523693,
   0.056702495,
   -0.02457165,
   0.017692043,
   -0.01827274,
   -0.059782285,
   0.01673569,
   -0.043824755,
   0.012825521,
   0.00961736,
   -0.1439678,
   -0.01956401,
   -0.14139986,
   0.051164687,
   -0.066762045,
   0.07247425,
   -0.014730731,
   0.09623298,
   0.08010232,
   0.01806106,
   -0.019820536,
   -0.062293377,
   0.07795393,
   -0.010080401,
   0.037061095,
   0.101677276,
   0.09910425,
   0.04335969,
   0.043769147,
   0.06242481,
   0.027671343,
   -0.047325373,
   0.057314157,
   0.058775067,
   -0.043589693,
   -0.0023683205,
   -0.022521196,
   0.029310443,
   0.03960663,
   -0.05888628,
   -0.0011037386,
   0.057607345,
   0.059740577,
   -0.014004779,
   0.010473747,
   -0.020675154,
   0.03357817,
   0.016248826,
   -0.070937514,
   -0.056641832,
   -0.04857587,
   -0.030350517,
   -0.008058066,
   -0.022976149,
   0.054905426,
 

In [5]:
import pyarrow as pa

# Define the schema for your table
schema = pa.schema([
    pa.field("id", pa.string()),
    pa.field("question", pa.string()),
    pa.field("answers", pa.list_(pa.string())),
    pa.field("vector", pa.list_(pa.float32(), list_size=1024))  # embedding column with dimension 1024
])

if table_name in db.table_names():
    table = db.open_table(table_name)
else:
    # Create the table with the specified schema
    table = db.create_table(table_name, schema=schema)

In [6]:
import numpy as np
def search_lancedb(query_embedding, top_k=1):
    query_embedding = np.array(query_embedding)
    results = table.search(query_embedding).limit(top_k).to_df()
    return results

In [19]:
query_embedding = get_embeddings('What is the capital of India')
data = query_embedding.get('data')
euclidean_threshold = 0.3

if(data is not None):
  # Retrieve the top document
  result = search_lancedb(data[0]['embedding'], top_k=1)

  print(result)

  # Print the result
  if not result.empty:
      distance = result.iloc[0]["_distance"]
      if distance <= euclidean_threshold:
        print("Most relevant document:", result.iloc[0]["question"])
      else:
        print("No relevant documents found.")
  else:
        print("No relevant documents found.")

{'model': 'jina-embeddings-v3', 'object': 'list', 'usage': {'total_tokens': 8, 'prompt_tokens': 8}, 'data': [{'object': 'embedding', 'index': 0, 'embedding': [0.00059130095, -0.058116283, 0.053348538, 0.0686821, -0.08204448, -0.0100289835, 0.053403374, -0.04372791, -0.13228475, -0.01829103, -0.077744275, -0.005256209, -0.009788562, 0.0013954027, -0.13231938, -0.16464308, 0.043521557, -0.038661458, -0.052973352, -0.01645307, -0.04817386, 0.019507408, -0.06347855, 0.0053247525, -0.041592233, -0.01757926, 0.04974964, 0.11417193, 0.028342403, -0.022094116, 0.022769451, -0.107002996, 0.102102496, -0.024906566, -0.049241696, -0.0044842823, 0.047642827, -0.041792814, 0.011522714, -0.02865121, 0.0132981725, 0.06259831, 0.01601615, -0.021457924, -0.009623697, -0.04113191, -0.020788902, -0.052064247, -0.024290215, -0.036485378, 0.03911745, 0.03095283, -0.029413126, -0.030043727, -0.071106374, -0.090338975, -0.042748094, -0.07379617, 0.09781383, -0.09210523, -0.009627845, -0.11844905, -0.02282139

<ipython-input-6-05f3a349165f>:4: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  results = table.search(query_embedding).limit(top_k).to_df()


# **DuckDuckGo Search**

Note - Please be aware about the DuckDuckGo Search Rate Limit. In such cases, try to run the below quite install. Give some gap and rerun

In [8]:
%pip install --upgrade --quiet duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.5 MB/s eta 0:00:00


In [9]:
from duckduckgo_search import DDGS
def search_with_duckduckgo(query, max_results=5):
    """
    Perform a real-time search using DuckDuckGo.

    Parameters:
        query (str): The search query.
        max_results (int): Maximum number of results to return.

    Returns:
        list: A list of search results, each containing title, link, and snippet.
    """
    try:
        results = DDGS().text(query, max_results=5)
        if len(results) > 0:
            return results
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [19]:
search_with_duckduckgo('What is the capital of India')

[{'title': 'New Delhi - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/New_Delhi',
  'body': 'New Delhi (/ ˈ nj uː ˈ d ɛ. l i / ⓘ, [6] Hindi: [ˈnəiː ˈdɪlːiː], ISO: Naī Dillī) is the capital of India and a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the Government of India, hosting the Rashtrapati Bhavan, Sansad Bhavan, and the Supreme Court.New Delhi is a municipality within the NCT, administered by the NDMC ...'},
 {'title': 'New Delhi | History, Population, Map, & Facts | Britannica',
  'href': 'https://www.britannica.com/place/New-Delhi',
  'body': 'New Delhi, national capital of India. It is situated in the north-central part of the country on the west bank of the Yamuna River, adjacent to and just south of Delhi city (Old Delhi) and within the Delhi national capital territory. Learn more about New Delhi in this article.'},
 {'title': 'List of state and union territory capitals in India - Wikipedia',
  'href': 'https:/

# **Semantic Realtime Search**

In [20]:
import json
import numpy as np
import time
import uuid

class SemanticSearchCacheFacade:
    def __init__(self, json_file='search-cache.json'):
        self.euclidean_threshold = 0.3
        self.json_file = json_file
        self.load_cache()

    def get_cache(self):
      return self.cache

    def load_cache(self):
        # Load cache from JSON file, creating an empty cache if the file is not found
        try:
            with open(self.json_file, 'r') as file:
                self.cache = json.load(file)
        except FileNotFoundError:
            self.cache = {'questions': [], 'embeddings': [], 'answers': []}

    def save_cache(self):
        # Save the cache to the JSON file
        with open(self.json_file, 'w') as file:
            json.dump(self.cache, file)

    def persist_in_cache(self, question, documents, embedding):
        # Persist a new question-answer pair in the cache
        self.cache['questions'].append(question)
        self.cache['embeddings'].append(embedding)
        self.cache['answers'].append(documents)
        item_to_add = [{"id": str(uuid.uuid4()), "question": question, "answers": [json.dumps(document) for document in documents], "vector": embedding}]
        print(item_to_add)
        table.add(item_to_add)
        self.save_cache()

    # Reused logic from Module:1a-Advanced-LLMs -semantic_cache_from_scratch.ipynb
    def search(self, question: str) -> str:
        # Method to retrieve an answer from the cache or generate a new one
        start_time = time.time()
        try:
            embeddings = get_embeddings(question)
            embedding = embeddings.get('data')[0]['embedding']

            # Search for the nearest neighbor in the index
            result = search_lancedb(embedding, top_k=1)

            if not result.empty:
                row_id = result.iloc[0]["id"]
                distance = result.iloc[0]["_distance"]
                print(f'Distance: {distance}')
                if distance <= self.euclidean_threshold:
                  print('RowId: '+ str(row_id))
                  print(f'Found cache in row: {row_id}')
                  end_time = time.time()
                  elapsed_time = end_time - start_time
                  print(f"Time taken: {elapsed_time} seconds")
                  return result.iloc[0]["answers"]

            # Handle the case when there are not enough results or Euclidean distance is not met
            documents = search_with_duckduckgo(question)

            if len(documents) > 0:
              self.persist_in_cache(question, documents, embedding)

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Time taken: {elapsed_time} seconds")

            return documents
        except Exception as e:
            raise RuntimeError(f"Error during 'ask' method: {e}")


In [21]:
cache_search_provider = SemanticSearchCacheFacade()
cache_search_provider.search('What is the capital of India')

{'model': 'jina-embeddings-v3', 'object': 'list', 'usage': {'total_tokens': 8, 'prompt_tokens': 8}, 'data': [{'object': 'embedding', 'index': 0, 'embedding': [0.00059130095, -0.058116283, 0.053348538, 0.0686821, -0.08204448, -0.0100289835, 0.053403374, -0.04372791, -0.13228475, -0.01829103, -0.077744275, -0.005256209, -0.009788562, 0.0013954027, -0.13231938, -0.16464308, 0.043521557, -0.038661458, -0.052973352, -0.01645307, -0.04817386, 0.019507408, -0.06347855, 0.0053247525, -0.041592233, -0.01757926, 0.04974964, 0.11417193, 0.028342403, -0.022094116, 0.022769451, -0.107002996, 0.102102496, -0.024906566, -0.049241696, -0.0044842823, 0.047642827, -0.041792814, 0.011522714, -0.02865121, 0.0132981725, 0.06259831, 0.01601615, -0.021457924, -0.009623697, -0.04113191, -0.020788902, -0.052064247, -0.024290215, -0.036485378, 0.03911745, 0.03095283, -0.029413126, -0.030043727, -0.071106374, -0.090338975, -0.042748094, -0.07379617, 0.09781383, -0.09210523, -0.009627845, -0.11844905, -0.02282139

<ipython-input-6-05f3a349165f>:4: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  results = table.search(query_embedding).limit(top_k).to_df()


array(['{"title": "New Delhi - Wikipedia", "href": "https://en.wikipedia.org/wiki/New_Delhi", "body": "New Delhi (/ \\u02c8 nj u\\u02d0 \\u02c8 d \\u025b. l i / \\u24d8, [6] Hindi: [\\u02c8n\\u0259i\\u02d0 \\u02c8d\\u026al\\u02d0i\\u02d0], ISO: Na\\u012b Dill\\u012b) is the capital of India and a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the Government of India, hosting the Rashtrapati Bhavan, Sansad Bhavan, and the Supreme Court.New Delhi is a municipality within the NCT, administered by the NDMC ..."}',
       '{"title": "List of state and union territory capitals in India - Wikipedia", "href": "https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India", "body": "India is a federal constitutional republic governed under a parliamentary system consisting of 28 states and 8 union territories. [1] All states, as well as the union territories of Jammu and Kashmir, Puducherry and the National Capital Terri

In [23]:
cache_search_provider.search('What is the capital of London')

{'model': 'jina-embeddings-v3', 'object': 'list', 'usage': {'total_tokens': 8, 'prompt_tokens': 8}, 'data': [{'object': 'embedding', 'index': 0, 'embedding': [-0.011421301, 0.03570107, 0.02254431, 0.040362265, 0.0034486763, -0.06807168, 0.035644595, -0.07085478, -0.07777635, -0.019003162, -0.030698141, 0.06876963, 0.054631107, -0.007964502, -0.123754054, -0.14724682, 0.042884722, -0.05360301, -0.07212325, -0.056759704, 0.033130825, 0.032729585, -0.014895296, -0.054749846, -0.064468995, -0.007376785, -0.010690501, 0.05603714, 0.026713176, 0.054631107, 0.11035113, -0.013689633, 0.12784328, -0.041900065, -0.0811821, 0.010242812, 0.032700762, -0.042933956, -0.03588207, -0.030545374, 0.062004454, 0.07824261, 0.001303223, -0.001188829, -0.0015493081, -0.03410824, -0.0013592434, -0.08911728, -0.054344397, 0.055847447, 0.05320625, 0.020447569, -0.08062027, -0.02518406, -0.08511494, -0.08753025, -0.005817171, -0.080683984, 0.09439968, -0.06745482, -0.033455905, -0.03990903, -0.007815718, -0.054

<ipython-input-6-05f3a349165f>:4: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  results = table.search(query_embedding).limit(top_k).to_df()


[{'id': 'f8a819ee-ceaa-4fee-a7bc-93bf0f6bde0e', 'question': 'What is the capital of London', 'answers': ['{"title": "London - Wikipedia", "href": "https://en.wikipedia.org/wiki/London", "body": "London [c] is the capital and largest city [d] of both England and the United Kingdom, with a population of 8,866,180 in 2022. [2] Its wider metropolitan area is the largest in Western Europe, with a population of 14.9 million. [7] London stands on the River Thames in southeast England, at the head of a 50-mile (80 km) tidal estuary down to the North Sea, and has been a major settlement for ..."}', '{"title": "London | History, Maps, Population, Area, & Facts | Britannica", "href": "https://www.britannica.com/place/London", "body": "London, city, capital of the United Kingdom. It is among the oldest of the world\'s great cities\\u2014its history spanning nearly two millennia\\u2014and one of the most cosmopolitan. By far Britain\'s largest metropolis, it is also the country\'s economic, transpo

[{'title': 'London - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/London',
  'body': 'London [c] is the capital and largest city [d] of both England and the United Kingdom, with a population of 8,866,180 in 2022. [2] Its wider metropolitan area is the largest in Western Europe, with a population of 14.9 million. [7] London stands on the River Thames in southeast England, at the head of a 50-mile (80 km) tidal estuary down to the North Sea, and has been a major settlement for ...'},
 {'title': 'London | History, Maps, Population, Area, & Facts | Britannica',
  'href': 'https://www.britannica.com/place/London',
  'body': "London, city, capital of the United Kingdom. It is among the oldest of the world's great cities—its history spanning nearly two millennia—and one of the most cosmopolitan. By far Britain's largest metropolis, it is also the country's economic, transportation, and cultural center. Learn more about London."},
 {'title': 'London - Simple English Wikipedia, the free